In [1]:
import pandas as pd , requests
import base64, json , os, uuid
import re

In [9]:
def extract_student_data(xl_file,school):
    student_names = []
    classes = []
    sections = []
    student_roll_nos = []
    student_ids = []
    for row in range(len(xl_file)):
        values = xl_file.iloc[row]
        # print(values)
        name = values["firstname"] + values["lastname"]
        Class = values["class"]
        section = values["section"]
        roll_no = values["rollnumber"]
        random_uuid = uuid.uuid4()
        # print(random_uuid)
        student_ids.append(random_uuid)
        student_names.append(name)
        classes.append(Class)
        sections.append(section)
        student_roll_nos.append(roll_no)
    data = {
        "student_id" : student_ids,
        "name":student_names,
        "class":classes,
        "section":sections,
        "roll_no":student_roll_nos,
        "school":[school]*len(xl_file)
    }
    df = pd.DataFrame(data)
    # Created unique ids for students of Class 1A
    df.to_csv(f"{Class}{section}_student_data.csv",index=False)

In [10]:
def extract_student_report(xl_file,audios_folder):
    audios = os.listdir(audios_folder)
    audio_exist = {}
    for audio in audios:
        audio_roll_no = audio.replace(".ogg","").replace(".mp3","")[::-1].split()[0][::-1]
        audio_exist[audio_roll_no] = f"{audios_folder}/{audio}"
    print(audio_exist)
    student_ids = []
    RCA = [] # Integer
    partA_scores = []
    partB_scores = []
    totals = []
    grades = []
    partA_reports = [] # dump the json here
    partB_reports = []
    feedbacks = []
    for row in range(len(xl_file)):
        id, name, Class, section, roll_no, school = xl_file.iloc[row].values
        if str(roll_no).strip() not in audio_exist:
            print("Audio not exist for", roll_no)
        else:
            audio_file_path = audio_exist[str(roll_no).strip()]
            print(roll_no, audio_file_path)
            with open(audio_file_path, "rb") as file:
                file_data = bytearray(file.read())

            file_base64 = base64.b64encode(file_data).decode('utf-8')
            print(len(file_base64))
            body = {
                "studentClass":int(Class),
                "audioBufferBase64":file_base64
            }
            report_response = requests.post("http://localhost:5000/api/get-report",json=body)
            report_json = report_response.json()
            partA_score = report_json["partA_score"]["score"]
            feedback = report_json["feedback"]
            print(partA_score,feedback)
            # id, RCA, partA_score, partB_score, total, grade, partA_report, feedback
            student_ids.append(id)
            RCA.append(1)
            partA_scores.append(partA_score)
            partB_scores.append(" ")
            partB_reports.append(" ")
            totals.append(" ")
            grades.append(" ")
            partA_reports.append(report_json)
            feedbacks.append(feedback)
            

    student_reports_data = {
        "student_id" : student_ids,
        "rca" : RCA,
        "partA_score":partA_scores,
        "partB_score":partB_scores,
        "total":totals,
        "grade":grades,
        "partA_report":partA_reports,
        "partB_report":partB_reports,
        "feedback":feedbacks
    }
    report_df = pd.DataFrame(student_reports_data)
    report_df.to_csv(f"{Class}{section}_student_audio_report.csv",index=False)

In [17]:
def final_report(partb_xl_file,student_report_file,student_data_file):
    partB_responses = {}
    part_b_roll_no = {}
    for row in range(len(partb_xl_file)):
        values = partb_xl_file.iloc[row]
        Class = values["class"]
        section = values["section"]
        roll_no = values["rollnumber"]
        part_b_roll_no[str(roll_no)] = 1
        response = [res.strip() for res in re.findall(r'\[([^\]]+)\]' , values["response"] ) ]
        marks = values["marks"]
        partB_responses[str(roll_no)] = {
            "marks":marks,
            "partB_report" :response
        }
    roll_no_mapped_student_id = {}
    for row in range(len(student_data_file)):
        data_values = student_data_file.iloc[row]
        roll_no_mapped_student_id[data_values["student_id"]] = str(roll_no)
    
    student_ids = [] 
    rcas = [] 
    partA_scores = []
    partB_scores = [] 
    totals = []
    grades = []
    partA_reports = []
    partB_reports = []
    feedbacks = []
    for row in range(len(student_report_file)):
        values = student_report_file.iloc[row]
        if roll_no_mapped_student_id[values["student_id"]] in part_b_roll_no:
            student_ids.append(values["student_id"])
            rcas.append(values["rca"])
            partA_scores.append(values["partA_score"])
            partB_scores.append(partB_responses[roll_no_mapped_student_id[values["student_id"]]]["marks"]*2)
            total = (values["partA_score"] + partB_responses[roll_no_mapped_student_id[values["student_id"]]]["marks"])/2
            totals.append(total)
            if total<50:
                grades.append("Emergent")
            elif total<80:
                grades.append("Transitional")
            else:
                grades.append("Proficient")
            partA_reports.append(json.dumps(values["partA_report"]))
            partB_reports.append('@'.join(partB_responses[roll_no_mapped_student_id[values["student_id"]]]["partB_report"]))
            feedbacks.append(json.dumps(values["feedback"]))
    data = {
        "student_id" : student_ids,
        "rca" : rcas,
        "partA_score" : partA_scores,
        "partB_score" : partB_scores,
        "total" : totals,
        "grade" : grades,
        "partA_report" : partA_reports,
        "partB_report" : partB_reports,
        "feedback" : feedbacks
    }

    class_1b_final_report = pd.DataFrame(data)
    class_1b_final_report.to_csv(f"{Class}{section}_student_final_report.csv",index=False)

Class 1B

In [26]:
class_1b_raw_excel = pd.read_csv("Class 1b.csv")
extract_student_data(class_1b_raw_excel,school="donbosco")

Class 1B Part A Report

In [ ]:
class_1b_student_data = pd.read_excel("1B_student_data.xlsx")
extract_student_report(class_1b_student_data,audios_folder="1 - B Recording list")

Class 1B- Part B report

Class 1A Final Report

In [18]:
class_1a_partb = pd.read_csv("Class - 1A-Part-B.csv")
class_1a_parta_report = pd.read_excel("1A_student_report_data.xlsx")
class_1a_student_data = pd.read_excel("1A_student_data.xlsx")
final_report(partb_xl_file=class_1a_partb,student_report_file=class_1a_parta_report,student_data_file=class_1a_student_data)